In [1]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import dlisio
import pandas as pd
import numpy as np
import scipy.signal as signal
import logging
import copy

# Ensure that we can see pandas tables of up to 500 rows
pd.set_option('display.max_rows', 500)

# Set a default DPI for all figures in this file. Lower DPI gives smaller figures, higher bigger.
mpl.rcParams['figure.dpi'] = 100

In [2]:
dlisio.set_encodings(['latin1'])

In [3]:
filename = 'MESTB _009_1B.dlis'

f, *f_tail = dlisio.load(filename)

if len(f_tail): logging.warning('There are more logical files in tail')

In [4]:
origin, *origin_tail = f.origins

if len(origin_tail): logging.warning('f contains multiple origins')

In [5]:
f.describe()

------------
Logical File
------------
Description : dlis(MESTB .009)
Frames      : 1
Channels    : 65

Known objects
--
TOOL        : 1
CHANNEL     : 65
FRAME       : 1
PARAMETER   : 423
ORIGIN      : 2
FILE-HEADER : 1


In [6]:
origin, *origin_tail = f.origins
print(len(origin_tail))

1


In [7]:
origin.describe()

------
Origin
------
name   : WELL-ODP Leg 194, Site 1196A
origin : 115
copy   : 0

Logical file ID          : MESTB .009
File set name and number : MESTB / 1
File number and type     : 0 / CUSTOMER

Well (id/name)          :  / ODP Leg 194, Site 1196A
Produced by (code/name) : 440 / Schlumberger
Produced for            : Lamont Doherty
Created                 : 2021-09-20 12:54:27

Created by              : Techlog, (version: 2017.2 (rev: 234070))
Other programs/services : WELL-ODP Leg 194, Site 1196A


In [8]:
def summarize(objs, **kwargs):
    """Create a pd.DataFrame that summarize the content of 'objs', One 
    object pr. row
    
    Parameters
    ----------
    
    objs : list()
        list of metadata objects
        
    **kwargs
        Keyword arguments 
        Use kwargs to tell summarize() which fields (attributes) of the 
        objects you want to include in the DataFrame. The parameter name 
        must match an attribute on the object in 'objs', while the value 
        of the parameters is used as a column name. Any kwargs are excepted, 
        but if the object does not have the requested attribute, 'KeyError' 
        is used as the value.
        
    Returns
    -------
    
    summary : pd.DataFrame
    """
    summary = []
    for attr, label in kwargs.items():
        column = []
        for obj in objs:
            try:
                value = getattr(obj, attr)
            except AttributeError:
                value = 'KeyError'
    
            column.append(value)
        summary.append(column)

    summary = pd.DataFrame(summary).T
    summary.columns = kwargs.values()
    return summary

In [9]:
for frame in f.frames:
    
    # Search through the channels for the index and obtain the units
    for channel in frame.channels:
        if channel.name == frame.index:
            depth_units = channel.units
    
    print(f'Frame Name: \t\t {frame.name}')
    print(f'Index Type: \t\t {frame.index_type}')
    print(f'Depth Interval: \t {frame.index_min} - {frame.index_max} {depth_units}')
    print(f'Depth Spacing: \t\t {frame.spacing} {depth_units}')
    print(f'Direction: \t\t {frame.direction}')
    print(f'Num of Channels: \t {len(frame.channels)}')
    print(f'Channel Names: \t\t {str(frame.channels)}')
    print('\n\n')

Frame Name: 		 1B
Index Type: 		 BOREHOLE-DEPTH
Depth Interval: 	 149101.0 - 322140.0 0.1 in
Depth Spacing: 		 -1.0 0.1 in
Direction: 		 DECREASING
Num of Channels: 	 65
Channel Names: 		 [Channel(TDEP), Channel(BA11), Channel(BA12), Channel(BA13), Channel(BA14), Channel(BA15), Channel(BA16), Channel(BA17), Channel(BA18), Channel(BA21), Channel(BA22), Channel(BA23), Channel(BA24), Channel(BA25), Channel(BA26), Channel(BA27), Channel(BA28), Channel(BB11), Channel(BB12), Channel(BB13), Channel(BB14), Channel(BB15), Channel(BB16), Channel(BB17), Channel(BB18), Channel(BB21), Channel(BB22), Channel(BB23), Channel(BB24), Channel(BB25), Channel(BB26), Channel(BB27), Channel(BB28), Channel(BC11), Channel(BC12), Channel(BC13), Channel(BC14), Channel(BC15), Channel(BC16), Channel(BC17), Channel(BC18), Channel(BC21), Channel(BC22), Channel(BC23), Channel(BC24), Channel(BC25), Channel(BC26), Channel(BC27), Channel(BC28), Channel(BD11), Channel(BD12), Channel(BD13), Channel(BD14), Channel(BD15), C

In [10]:
def summary_dataframe(object, **kwargs):
    # Create an empty dataframe
    df = pd.DataFrame()
    
    # Iterate over each of the keyword arguments
    for i, (key, value) in enumerate(kwargs.items()):
        list_of_values = []
        
        # Iterate over each parameter and get the relevant key
        for item in object:
            # Account for any missing values.
            try:
                x = getattr(item, key)
                list_of_values.append(x)
            except:
                list_of_values.append('')
                continue
        
        # Add a new column to our data frame
        df[value]=list_of_values
    
    # Sort the dataframe by column 1 and return it
    return df.sort_values(df.columns[0])


In [11]:
param_df = summary_dataframe(f.parameters, name='Name', long_name='Long Name', values='Value')

# Hiding people's names that may be in parameters.
# These two lines can be commented out to show them
mask = param_df['Name'].isin(['R8', 'RR1', 'WITN', 'ENGI'])
param_df = param_df[~mask]

param_df

,Name,Long Name,Value
415,ACCC,Accelerometer coefficients,[-0.0004;0;0.0011;-0.0003;2e-06;-7.6e-05;1e-06...
416,ACCM,Accelerometer Month of Calibration,[10]
405,ACCN,Accelerometer Serial Number,[448]
419,ACCT,Accelerometer Type (Manufacturer Code),[3]
406,ACCY,Accelerometer Year of Calibration,[92]
409,ACPP,Accelerometer PROM Presence,[PRESENT]
372,ACQUISITION-PRODUCT,,[OP]
414,AFMO,Accelerometer Filtering Mode,[MAVE]
26,ALTDPCHAN,Name of alternate depth channel,[SPEEDCORRECTEDDEPTH]
196,ALTDPCHAN,Name of alternate depth channel,[SPEEDCORRECTEDDEPTH]


In [12]:
channels = summary_dataframe(f.channels, name='Name', long_name='Long Name', dimension='Dimension', units='Units', frame='Frame')
channels

,Name,Long Name,Dimension,Units,Frame
1,BA11,CALIBRATED DATA BUTTON A11,[1],,Frame(1B)
2,BA12,CALIBRATED DATA BUTTON A12,[1],,Frame(1B)
3,BA13,CALIBRATED DATA BUTTON A13,[1],,Frame(1B)
4,BA14,CALIBRATED DATA BUTTON A14,[1],,Frame(1B)
5,BA15,CALIBRATED DATA BUTTON A15,[1],,Frame(1B)
6,BA16,CALIBRATED DATA BUTTON A16,[1],,Frame(1B)
7,BA17,CALIBRATED DATA BUTTON A17,[1],,Frame(1B)
8,BA18,CALIBRATED DATA BUTTON A18,[1],,Frame(1B)
9,BA21,CALIBRATED DATA BUTTON A21,[1],,Frame(1B)
10,BA22,CALIBRATED DATA BUTTON A22,[1],,Frame(1B)


In [13]:
frame1 =f.object('FRAME','1B') 

In [14]:
curves = frame1.curves()

In [15]:
df = pd.DataFrame(frame.curves())
df

,FRAMENO,TDEP,BA11,BA12,BA13,BA14,BA15,BA16,BA17,BA18,...,BD17,BD18,BD21,BD22,BD23,BD24,BD25,BD26,BD27,BD28
0,1,322140.0,-0.131836,-0.136719,-0.039062,-0.161133,-0.180664,-0.141602,-0.117188,-0.112305,...,0.092773,0.185547,-0.043945,-0.014648,0.039062,-0.024414,0.068359,0.092773,0.180664,0.141602
1,2,322139.0,-0.131836,-0.136719,-0.039062,-0.161133,-0.180664,-0.141602,-0.117188,-0.112305,...,0.092773,0.185547,-0.043945,-0.014648,0.039062,-0.024414,0.068359,0.092773,0.180664,0.141602
2,3,322138.0,-0.131836,-0.136719,-0.039062,-0.161133,-0.180664,-0.141602,-0.117188,-0.112305,...,0.092773,0.185547,-0.043945,-0.014648,0.039062,-0.024414,0.068359,0.092773,0.180664,0.141602
3,4,322137.0,-0.131836,-0.136719,-0.039062,-0.161133,-0.180664,-0.141602,-0.117188,-0.112305,...,0.092773,0.185547,-0.043945,-0.014648,0.039062,-0.024414,0.068359,0.092773,0.180664,0.141602
4,5,322136.0,-0.131836,-0.136719,-0.039062,-0.161133,-0.180664,-0.141602,-0.117188,-0.112305,...,0.092773,0.185547,-0.043945,-0.014648,0.039062,-0.024414,0.068359,0.092773,0.180664,0.141602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173035,173036,149105.0,27.504883,20.004883,16.250000,14.223633,13.520508,13.090820,13.786621,14.965820,...,7.338867,7.587891,9.741211,8.754883,8.007812,7.519531,7.377930,7.426758,7.705078,7.258301
173036,173037,149104.0,27.504883,20.004883,16.250000,14.223633,13.520508,13.090820,13.786621,14.965820,...,7.338867,7.587891,9.741211,8.754883,8.007812,7.519531,7.377930,7.426758,7.705078,7.258301
173037,173038,149103.0,27.504883,20.004883,16.250000,14.223633,13.520508,13.090820,13.786621,14.965820,...,7.338867,7.587891,9.741211,8.754883,8.007812,7.519531,7.377930,7.426758,7.705078,7.258301
173038,173039,149102.0,27.504883,20.004883,16.250000,14.223633,13.520508,13.090820,13.786621,14.965820,...,7.338867,7.587891,9.741211,8.754883,8.007812,7.519531,7.377930,7.426758,7.705078,7.258301


In [16]:
df["TDEP"]=(df["TDEP"])* 0.00254

In [18]:
df.drop("FRAMENO", axis=1, inplace=True)
df

,TDEP,BA11,BA12,BA13,BA14,BA15,BA16,BA17,BA18,BA21,...,BD17,BD18,BD21,BD22,BD23,BD24,BD25,BD26,BD27,BD28
0,818.23560,-0.131836,-0.136719,-0.039062,-0.161133,-0.180664,-0.141602,-0.117188,-0.112305,-0.136719,...,0.092773,0.185547,-0.043945,-0.014648,0.039062,-0.024414,0.068359,0.092773,0.180664,0.141602
1,818.23306,-0.131836,-0.136719,-0.039062,-0.161133,-0.180664,-0.141602,-0.117188,-0.112305,-0.136719,...,0.092773,0.185547,-0.043945,-0.014648,0.039062,-0.024414,0.068359,0.092773,0.180664,0.141602
2,818.23052,-0.131836,-0.136719,-0.039062,-0.161133,-0.180664,-0.141602,-0.117188,-0.112305,-0.136719,...,0.092773,0.185547,-0.043945,-0.014648,0.039062,-0.024414,0.068359,0.092773,0.180664,0.141602
3,818.22798,-0.131836,-0.136719,-0.039062,-0.161133,-0.180664,-0.141602,-0.117188,-0.112305,-0.136719,...,0.092773,0.185547,-0.043945,-0.014648,0.039062,-0.024414,0.068359,0.092773,0.180664,0.141602
4,818.22544,-0.131836,-0.136719,-0.039062,-0.161133,-0.180664,-0.141602,-0.117188,-0.112305,-0.136719,...,0.092773,0.185547,-0.043945,-0.014648,0.039062,-0.024414,0.068359,0.092773,0.180664,0.141602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173035,378.72670,27.504883,20.004883,16.250000,14.223633,13.520508,13.090820,13.786621,14.965820,21.762695,...,7.338867,7.587891,9.741211,8.754883,8.007812,7.519531,7.377930,7.426758,7.705078,7.258301
173036,378.72416,27.504883,20.004883,16.250000,14.223633,13.520508,13.090820,13.786621,14.965820,21.762695,...,7.338867,7.587891,9.741211,8.754883,8.007812,7.519531,7.377930,7.426758,7.705078,7.258301
173037,378.72162,27.504883,20.004883,16.250000,14.223633,13.520508,13.090820,13.786621,14.965820,21.762695,...,7.338867,7.587891,9.741211,8.754883,8.007812,7.519531,7.377930,7.426758,7.705078,7.258301
173038,378.71908,27.504883,20.004883,16.250000,14.223633,13.520508,13.090820,13.786621,14.965820,21.762695,...,7.338867,7.587891,9.741211,8.754883,8.007812,7.519531,7.377930,7.426758,7.705078,7.258301


In [21]:
cols=df.columns
cols

Index(['TDEP', 'BA11', 'BA12', 'BA13', 'BA14', 'BA15', 'BA16', 'BA17', 'BA18',
       'BA21', 'BA22', 'BA23', 'BA24', 'BA25', 'BA26', 'BA27', 'BA28', 'BB11',
       'BB12', 'BB13', 'BB14', 'BB15', 'BB16', 'BB17', 'BB18', 'BB21', 'BB22',
       'BB23', 'BB24', 'BB25', 'BB26', 'BB27', 'BB28', 'BC11', 'BC12', 'BC13',
       'BC14', 'BC15', 'BC16', 'BC17', 'BC18', 'BC21', 'BC22', 'BC23', 'BC24',
       'BC25', 'BC26', 'BC27', 'BC28', 'BD11', 'BD12', 'BD13', 'BD14', 'BD15',
       'BD16', 'BD17', 'BD18', 'BD21', 'BD22', 'BD23', 'BD24', 'BD25', 'BD26',
       'BD27', 'BD28'],
      dtype='object')

In [22]:
fmsdf = df[['BA11', 'BA12', 'BA13', 'BA14', 'BA15', 
               'BA16', 'BA17', 'BA18', 'BA21','BA22', 
               'BA23', 'BA24', 'BA25', 'BA26', 'BA27', 
               'BA28', 'BB11', 'BB12', 'BB13', 'BB14', 'BB15', 'BB16', 'BB17',
               'BB18', 'BB21', 'BB22', 'BB23', 'BB24', 'BB25', 'BB26', 'BB27', 'BB28',
               'BC11', 'BC12', 'BC13', 'BC14', 'BC15', 'BC16', 'BC17', 'BC18', 'BC21',
               'BC22', 'BC23', 'BC24', 'BC25', 'BC26', 'BC27', 'BC28', 'BD11', 'BD12',
               'BD13', 'BD14', 'BD15', 'BD16', 'BD17', 'BD18', 'BD21', 'BD22', 'BD23',
               'BD24', 'BD25', 'BD26', 'BD27', 'BD28']]

In [38]:
fms1 = ['BA11', 'BA12', 'BA13', 'BA14', 'BA15', 'BA16', 'BA17', 'BA18', 
             'BA21','BA22', 'BA23', 'BA24', 'BA25', 'BA26', 'BA27', 'BA28']

fms2 = ['BB11', 'BB12', 'BB13', 'BB14', 'BB15', 'BB16', 'BB17', 'BB18', 
             'BB21','BB22', 'BB23', 'BB24', 'BB25', 'BB26', 'BB27', 'BB28']

fms3 = ['BC11', 'BC12', 'BC13', 'BC14', 'BC15', 'BC16', 'BC17', 'BC18', 
             'BC21','BC22', 'BC23', 'BC24', 'BC25', 'BC26', 'BC27', 'BC28']


fms123 = [fms1,fms2,fms3]

In [40]:
normfms_list = []

for fms in fms123:
    fmsnorm = df[fms].copy()
    fmsnorm = ((fmsnorm-fmsnorm.min())/(fmsnorm.max()-fmsnorm.min()))*255
    normfms_list.append(fmsnorm)

allnorm = pd.concat(normfms_list,axis=1)
allnorm["TDEP"]=df["TDEP"].copy()
allnorm

,BA11,BA12,BA13,BA14,BA15,BA16,BA17,BA18,BA21,BA22,...,BC18,BC21,BC22,BC23,BC24,BC25,BC26,BC27,BC28,TDEP
0,1.917413,0.784441,3.472436,2.313603,1.486626,1.434395,1.198656,2.542006,2.688807,1.045906,...,2.673699,3.033480,2.751192,3.048043,3.379519,4.059012,3.607037,5.591616,21.813217,818.23560
1,1.917413,0.784441,3.472436,2.313603,1.486626,1.434395,1.198656,2.542006,2.688807,1.045906,...,2.673699,3.033480,2.751192,3.048043,3.379519,4.059012,3.607037,5.591616,21.813217,818.23306
2,1.917413,0.784441,3.472436,2.313603,1.486626,1.434395,1.198656,2.542006,2.688807,1.045906,...,2.673699,3.033480,2.751192,3.048043,3.379519,4.059012,3.607037,5.591616,21.813217,818.23052
3,1.917413,0.784441,3.472436,2.313603,1.486626,1.434395,1.198656,2.542006,2.688807,1.045906,...,2.673699,3.033480,2.751192,3.048043,3.379519,4.059012,3.607037,5.591616,21.813217,818.22798
4,1.917413,0.784441,3.472436,2.313603,1.486626,1.434395,1.198656,2.542006,2.688807,1.045906,...,2.673699,3.033480,2.751192,3.048043,3.379519,4.059012,3.607037,5.591616,21.813217,818.22544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173035,92.607338,67.273941,56.855133,50.425682,47.665245,45.110920,47.013062,51.911285,74.328201,61.805016,...,35.893600,28.188328,28.075769,29.850119,31.168377,32.366547,31.821009,36.488880,52.977077,378.72670
173036,92.607338,67.273941,56.855133,50.425682,47.665245,45.110920,47.013062,51.911285,74.328201,61.805016,...,35.893600,28.188328,28.075769,29.850119,31.168377,32.366547,31.821009,36.488880,52.977077,378.72416
173037,92.607338,67.273941,56.855133,50.425682,47.665245,45.110920,47.013062,51.911285,74.328201,61.805016,...,35.893600,28.188328,28.075769,29.850119,31.168377,32.366547,31.821009,36.488880,52.977077,378.72162
173038,92.607338,67.273941,56.855133,50.425682,47.665245,45.110920,47.013062,51.911285,74.328201,61.805016,...,35.893600,28.188328,28.075769,29.850119,31.168377,32.366547,31.821009,36.488880,52.977077,378.71908


In [47]:
intervals = pd.read_csv('1196A_coreinfo.csv')
intervals = intervals[['leg', 'site', 'hole', 'core', 't', 'sc', 'min_depth', 'max_depth',
       'core_min', 'core_max']].copy()
intervals

,leg,site,hole,core,t,sc,min_depth,max_depth,core_min,core_max
0,194.0,1196.0,A,10.0,R,1.0,399.575,399.610,7.5,11.0
1,194.0,1196.0,A,10.0,R,1.0,399.620,399.660,12.0,16.0
2,194.0,1196.0,A,10.0,R,1.0,399.720,399.750,22.0,25.0
3,194.0,1196.0,A,10.0,R,1.0,399.770,399.800,27.0,30.0
4,194.0,1196.0,A,10.0,R,1.0,399.850,399.880,35.0,38.0
5,194.0,1196.0,A,10.0,R,1.0,399.890,399.920,39.0,42.0
6,194.0,1196.0,A,10.0,R,1.0,399.980,400.010,48.0,51.0
7,194.0,1196.0,A,10.0,R,1.0,400.150,400.180,65.0,68.0
8,194.0,1196.0,A,10.0,R,1.0,400.200,400.230,70.0,73.0
9,194.0,1196.0,A,10.0,R,1.0,400.250,400.280,75.0,78.0


In [49]:
for index, interval in intervals.iterrows():
    min_depth = interval.min_depth
    max_depth = interval.max_depth
    print(allnorm[(allnorm['TDEP']>=min_depth) & (allnorm['TDEP']<=max_depth)])

             BA11       BA12       BA13       BA14       BA15       BA16  \
164814  26.528671  24.833492  26.723354  25.912363  25.629190  25.625711   
164815  26.833107  25.075270  26.931379  26.173664  25.859591  25.819113   
164816  27.153566  25.429882  27.283422  26.516623  26.221645  26.189798   
164817  27.393911  25.703899  27.523455  26.794254  26.435589  26.399317   
164818  27.650278  25.913441  27.795488  26.990231  26.649530  26.560486   
164819  27.794485  25.994034  27.827492  27.022894  26.698902  26.608835   
164820  27.834541  26.050449  27.827492  27.047392  26.690672  26.600777   
164821  27.882610  26.114925  27.907501  27.063723  26.707130  26.633011   
164822  27.866589  26.098806  27.907501  27.096384  26.740046  26.633011   
164823  27.930681  26.163280  27.971510  27.129047  26.740046  26.665243   
164824  28.106932  26.292231  28.083523  27.259697  26.888159  26.794178   
164825  28.074886  26.292231  28.099525  27.259697  26.871702  26.778061   
164826  28.0

             BA11       BA12       BA13       BA14       BA15       BA16  \
160688  30.854870  26.574306  26.131279  23.005388  20.017321  17.019341   
160689  30.854870  26.558189  26.083273  22.972725  20.000862  17.019341   
160690  30.822823  26.574306  26.099276  22.989058  20.000862  16.987106   
160691  30.854870  26.558189  26.083273  23.005388  20.017321  17.035458   
160692  30.854870  26.558189  26.099276  22.972725  20.033777  17.035458   
160693  30.838846  26.542070  26.067272  22.940063  20.017321  17.051573   
160694  30.870892  26.574306  26.099276  22.940063  20.000862  17.083807   
160695  30.822823  26.542070  26.043268  22.891069  20.000862  17.132158   
160696  30.806801  26.542070  26.043268  22.874739  19.935034  17.132158   
160697  30.790777  26.493713  25.963259  22.793081  19.836292  17.067690   
160698  30.806801  26.477594  25.931255  22.760420  19.754005  16.874289   
160699  30.774755  26.461475  25.915251  22.727757  19.671721  16.680889   

           

             BA11       BA12       BA13       BA14       BA15       BA16  \
152796  24.069149  15.218707  16.994133  12.708489  10.315860   7.937524   
152797  25.575306  16.862812  20.066519  18.571436  15.170704   9.645904   
152798  27.337831  18.345728  21.074644  21.102789  15.532760  10.886898   
152799  28.843988  19.038832  18.354303  16.611675  10.891858   7.566837   
152800  29.484907  18.893763  14.529823   8.756310   6.037014   9.371919   
152801  29.244562  17.926641  12.321547   5.914660   5.411644  15.077266   
152802  28.539553  16.637150  11.201406   7.025189   9.427176  20.847080   
152803  28.715805  17.346371  11.185404   9.164593  12.208426  19.944538   
152804  30.302078  19.538509  12.113520   9.115599  13.031281  17.494785   
152805  31.319536  20.529808  13.857738  10.887547  14.257335  25.706295   
152806  32.280914  22.415691  15.681968  14.251800  15.359961  31.959614   
152807  32.809669  25.107508  16.370054  10.789559  17.170240  24.868221   
152808  31.8

             BA11       BA12       BA13       BA14       BA15       BA16  \
149207  54.512760  46.231022  43.637478  39.949139  38.062527  37.463501   
149208  54.384579  46.231022  43.893509  40.275764  38.556244  38.092056   
149209  54.384579  46.359970  44.149544  40.863689  38.967670  38.688377   
149210  54.384579  46.359970  44.277557  41.124992  39.412010  39.284698   
149211  54.512760  46.617870  44.533588  41.386292  39.856354  40.058304   
149212  54.512760  46.746819  44.789623  41.778244  40.531094  40.574043   
149213  54.512760  46.875767  44.917637  41.908894  40.794407  40.960846   
149214  54.769127  47.262615  45.429703  42.692799  41.584351  41.734451   
149215  55.025494  47.520515  45.813751  43.084747  41.979321  42.250191   
149216  55.538231  48.036312  46.325813  43.476700  42.505943  42.765926   
149217  55.538231  48.036312  46.453831  43.738003  42.769260  43.023796   
149218  55.538231  48.165260  46.453831  43.868652  43.032574  43.281666   

           

             BA11       BA12       BA13       BA14       BA15       BA16  \
133922  25.367008  15.686149  13.713720  10.961039   8.719521  11.918373   
133923  25.527237  16.379251  12.881617   9.393230   7.468781  12.804797   
133924  25.655422  17.523676  14.481817  11.646954  10.908315  11.579921   
133925  25.687468  18.055592  15.986007  13.737363  11.780541  11.467103   
133926  26.120087  18.232899  16.290045  12.969790  11.813455  10.806314   
133927  26.921234  18.442440  15.409933  11.303996  10.365232  13.530053   
133928  28.603645  19.393440  15.137899  11.712278  10.776658  13.965206   
133929  29.965595  20.263849  16.034014  12.430858  10.974144  14.464828   
133930  28.443415  19.506271  16.658089  13.198429  11.862826  11.418752   
133931  26.841120  18.603626  17.330175  14.325291  13.952878  13.820156   
133932  27.642267  19.361204  18.690346  17.869186  15.861902  13.030433   
133933  28.074886  19.490154  18.594332  15.403157  14.051621  10.403395   

           

...

            BA11      BA12       BA13       BA14       BA15       BA16  \
35678   7.421298  7.062662  11.081391  11.255002  10.463973   7.937524   
35679   7.453344  6.949831  10.889366  11.140683  10.332316   7.695772   
35680   7.453344  6.885356  10.793354  11.026363  10.233574   7.582954   
35681   7.437321  6.853118  10.745348  11.026363  10.266488   7.518487   
35682   7.357206  7.094899  11.129396  11.287664  10.529802   7.840823   
35683   7.229023  7.465628  11.961501  11.908256  11.056429   8.823947   
35684   7.212999  7.642933  12.393556  12.218551  11.369113   9.371919   
35685   7.188965  7.731586  12.678925  12.422691  11.591285   9.895715   
35686   7.156919  7.747705  13.094978  12.781981  11.920426  10.669321   
35687   7.221011  7.957248  13.415018  13.010620  12.150826  11.233410   
35688   7.301126  8.037841  13.527032  13.092276  12.233111  11.426810   
35689   7.301126  8.199027  13.639046  13.206595  12.331855  11.555745   
35690   7.477378  8.730944  14.007092 

           BA11       BA12       BA13       BA14       BA15       BA16  \
9459  58.230087  57.707508  59.903515  60.461273  61.859497  62.879696   
9460  58.614635  58.094357  60.287563  60.591927  62.386124  63.395432   
9461  59.255554  58.868053  60.927643  61.637127  63.176064  64.169037   
9462  61.178310  60.415443  62.591858  63.204933  65.019264  66.489861   
9463  66.305649  64.283928  66.048294  66.340546  68.310684  69.584282   
9464  67.972046  66.089218  67.840508  68.169655  69.758904  71.002556   
9465  69.251213  67.376022  68.355240  68.428238  70.151138  71.386681   
9466  68.610298  66.860229  68.099205  68.558891  70.414444  71.902420   
9467  69.251213  67.504974  68.867310  69.604095  71.467705  72.933891   
9468  68.738480  67.247070  69.123337  70.126694  72.257645  73.965370   
9469  67.328461  66.473381  68.867310  70.388000  73.047585  75.254707   
9470  66.687538  65.957581  68.611267  70.257347  73.179237  75.383644   

           BA17       BA18       BA21

           BA11       BA12       BA13       BA14       BA15       BA16  \
8658  32.921833  27.122343  23.869661  17.959011  12.743281   8.541903   
8659  31.591927  26.251934  24.013680  18.530605  14.010478  10.282518   
8660  25.262859  20.932775  20.285212  15.770613  12.068540   8.880356   
8661   2.974928   2.073934   4.571239   3.277151   2.441138   2.014600   
8662   4.513131   2.783155   4.587241   2.917862   1.634740   1.079826   
8663   2.446170   2.525257   4.363213   2.280941   1.009370   1.031475   
8664   1.869344   0.897271   3.243073   1.807332   1.108112   1.514979   
8665   2.334009   1.090695   3.515107   1.954314   1.141027   1.176526   
8666   6.596116   4.749632   6.043424   3.277151   2.062624   1.934016   
8667   9.448201   6.425974   7.307583   4.632650   3.560220   3.465112   
8668  11.499140   7.667111   8.379717   5.988151   4.876788   4.689989   
8669  13.197574   8.698706   9.355840   7.098680   5.995871   5.673113   
8670  13.181550   8.682588   9.345173 

           BA11       BA12       BA13       BA14       BA15       BA16  \
7942  17.641273  14.023240  14.465816  12.248491  11.020773  10.513527   
7943  17.448997  14.087714  14.705846  12.640444  11.711971  11.754520   
7944  17.272747  14.007121  14.577829  12.444468  11.382829  11.416067   
7945  17.240700  13.878172  14.145776  11.317606   9.111750   7.531919   
7946  17.368883  13.862053  13.873741  10.729679   7.959753   5.791305   
7947  17.673319  14.136071  14.065765  10.827667   7.861010   5.517319   
7948  17.753433  14.216663  14.193781  10.925654   7.959753   5.565669   
7949  17.817528  14.281138  14.225785  10.974649   7.976210   5.581786   
7950  17.956390  14.420833  14.465816  11.230505   8.266951   5.785931   
7951  18.132645  14.630375  14.657840  11.361156   8.464436   5.979332   
7952  18.180712  14.662612  14.721848  11.475474   8.579636   6.076034   
7953  18.180712  14.678731  14.465816  11.361156   8.530264   6.076034   
7954  18.196735  14.710969  14.673841 

          BA11      BA12       BA13       BA14      BA15       BA16  \
5243  2.601058  2.278103   4.720592   4.414898  1.826738   1.799708   
5244  3.033677  1.987967   4.880612   4.333242  2.221709   1.686891   
5245  3.834825  3.309697   3.984500   3.010405  3.521819   2.428263   
5246  3.017654  4.018919   4.624580   2.765435  2.600222   2.718366   
5247  3.225953  2.938968   6.176775   2.879754  2.666050   1.799708   
5248  4.107215  2.503764   4.976624   4.659869  3.225591   5.990076   
5249  5.100639  3.084036   4.976624   4.578212  5.546042  10.099860   
5250  5.402405  4.096827   5.344671   6.763888  8.341007   5.391069   
5251  4.697395  4.403081   8.193028  11.516303  5.000216   1.780906   
5252  5.065923  4.435318  11.601456  10.830388  2.663308   1.684206   
5253  5.113992  4.451437  12.033509  10.634412  2.498737   1.651972   
5254  5.113992  4.451437  12.017508  10.634412  2.465823   1.651972   
5255  5.097970  4.483675  12.049513  10.634412  2.449366   1.684206   
5256  